# Ex 1 :
Nous devons résoudre le problème en faisant la comparaison entre A* et un algorithme de notre choix. Nous avons pris BFS, car c’est un algorithme simple mais tout de même optimal.

In [3]:
import heapq
import time
import math

graph_ex = {
    (0,0) : [((1,0),1), ((0,1),1)],
    (0,1) : [((0,0),1), ((1,1),1)],
    (0,3) : [((1,3),1), ((0,4),math.inf)],
    
    (1,0) : [((0,0),1), ((1,1),1)],
    (1,1) : [((0,1),1), ((1,0),1), ((2,1),1), ((1,2),1)],
    (1,2) : [((1,3),1), ((1,1),1)],
    (1,3) : [((1,2),1), ((1,4),1), ((0,3),1)],
    (1,4) : [((0,4),math.inf), ((2,4),1)],
    
    (2,1) : [((1,1),1), ((3,1),1)],
    (2,4) : [((1,4),1), ((3,4),1)],

    (3,0) : [((3,1),1), ((4,0),1)],
    (3,1) : [((3,0),1), ((2,1),1), ((4,1),1), ((3,2), math.inf)],
    (3,3) : [((3,4),1), ((4,3),1), ((3,2),math.inf)],
    (3,4) : [((2,4),1), ((3,3),1), ((4,4),1)],

    (4,0) : [((3,0),1), ((4,1),1)],
    (4,1) : [((4,0),1), ((3,1),1), ((4,2),1)],
    (4,2) : [((4,1),1), ((3,2),math.inf), ((4,3),1)],
    (4,3) : [((3,3),1), ((4,4),1), ((4,2),1)],
    (4,4) : [((4,4),1), ((4,3),1), ((3,4),1)],
}

# Breadth-First Search (BFS)
def bfs(graph, start, goal):
    # La queue contient tous les chemins que nous pouvons explorer.
    # Pour chaque élément de la queue, nous stockons :
    # - le nœud actuel à partir duquel nous devons explorer
    # - le chemin complet parcouru jusqu'à ce nœud
    # - le coût cumulé du chemin jusqu'à ce nœud
    queue = [(start, [start], 0)]  # Initialisation de la queue avec le nœud de départ, le chemin initial (départ), et le coût (0)
    visited = set()  # Ensemble pour suivre les nœuds déjà visités afin d'éviter les boucles infinies

    while queue:  # Tant qu'il y a des nœuds à explorer dans la queue
        # Trier la queue par ordre alphabétique des nœuds
        queue.sort()  
        
        # Récupérer et enlever le premier élément de la queue
        # node : le nœud actuel à explorer
        # path : le chemin suivi pour atteindre ce nœud
        # cost : le coût total pour atteindre ce nœud
        node, path, cost = queue.pop(0)

        # Si nous avons trouvé le but, retourner le chemin et le coût
        if node == goal:
            return path, cost

        # Si le nœud n'a pas encore été visité
        if node not in visited:
            visited.add(node)  # Marquer le nœud comme visité
            # Explorer tous les voisins du nœud actuel
            for neighbor, weight in graph.get(node, []):  # Pour chaque voisin et le poids de l'arête
                # Créer un nouveau chemin incluant le voisin
                new_path = path + [neighbor]
                # Ajouter le nouveau chemin à la queue avec le coût mis à jour
                queue.append((neighbor, new_path, cost + weight))

    return None  # Si aucun chemin n'a été trouvé


In [4]:
def heuristic(node, goal):
    """Calcul de la distance de Manhattan (heuristique) entre le nœud actuel et le but."""
    # La distance de Manhattan est la somme des différences absolues des coordonnées
    return abs(node[0] - goal[0]) + abs(node[1] - goal[1])

def trier_la_queue(queue, goal):
    """Trie la queue selon l'heuristique (distance de Manhattan) + coût cumulé."""
    new_queue = []  # Liste vide pour stocker la queue triée
    while queue:
        value_heuristic = math.inf  # Valeur initiale de l'heuristique (très grande)
        best_position = 0  # Position du meilleur nœud à explorer
        position = 0  # Position dans la queue
        for node in queue:
            # Calcul de l'heuristique pour le nœud actuel (en tenant compte du coût)
            node_value_heuristic = heuristic(node[0], goal) + node[2]
            if node_value_heuristic < value_heuristic:  # Si cet nœud est meilleur
                value_heuristic = node_value_heuristic  # Mettre à jour la meilleure heuristique
                best_position = position  # Mettre à jour la meilleure position
            position += 1
        # Ajouter le meilleur nœud à la nouvelle queue
        new_queue.append(queue[best_position])
        # Retirer ce nœud de la queue initiale
        queue.pop(best_position)
    return new_queue  # Retourner la queue triée

def astar_1(graph, start, goal):
    """Recherche de chemin avec BFS en utilisant une heuristique (distance de Manhattan)."""
    # La queue contient les chemins à explorer : (nœud actuel, chemin suivi, coût cumulé)
    queue = [(start, [start], 0)]
    visited = set()  # Ensemble pour suivre les nœuds déjà visités
    while queue:
        # Trier la queue en fonction de l'heuristique
        queue = trier_la_queue(queue, goal)
        # Récupérer et enlever le premier élément de la queue
        node, path, cost = queue.pop(0)
        # Si le nœud actuel est l'objectif, retourner le chemin et le coût
        if node == goal:
            return path, cost
        # Si le nœud n'a pas été visité
        if node not in visited:
            visited.add(node)  # Marquer ce nœud comme visité
            # Explorer tous les voisins du nœud actuel
            for neighbor, weight in graph.get(node, []):
                # Créer un nouveau chemin incluant ce voisin
                new_path = path + [neighbor]
                # Ajouter ce nouveau chemin à la queue avec son coût mis à jour
                queue.append((neighbor, new_path, cost + weight))
    return None  # Si aucun chemin n'a été trouvé


In [13]:
"""
A* Search avec heapq inspiré de ChatGpt 
"""

def calculate_cost(graph, path):
    """Calcule le coût total d'un chemin en sommant les coûts des arêtes."""
    cost = 0
    for i in range(len(path) - 1):
        current, next_node = path[i], path[i + 1]
        for neighbor, weight in graph[current]:
            if neighbor == next_node:
                cost += weight
                break
    return cost

def astar_2(graph, start, goal):
    """Recherche de chemin avec A* en utilisant `calculate_cost()` pour le coût final."""
    open_set = []
    heapq.heappush(open_set, (0, start, [start]))  # (f_score, node, path)
    g_scores = {start: 0}

    while open_set:
        _, current, path = heapq.heappop(open_set)

        if current == goal:
            return path, calculate_cost(graph, path)  # ✅ On calcule le coût après coup

        for neighbor, weight in graph.get(current, []):
            if weight == math.inf:
                continue  # Ignorer les chemins impossibles

            tentative_g_score = g_scores[current] + weight

            if neighbor not in g_scores or tentative_g_score < g_scores[neighbor]:
                g_scores[neighbor] = tentative_g_score
                new_path = path + [neighbor]
                f_score = tentative_g_score + heuristic(neighbor, goal)
                heapq.heappush(open_set, (f_score, neighbor, new_path))

    return None, math.inf  # Aucun chemin trouvé


#nous comparons
start, goal = (0, 0), (3, 4)

# Trier la queue sans heapq
start_time = time.time()
astar_1_path, astar_1_cost = astar_1(graph_ex, start=(0, 0), goal=(3, 4))
astar_1_time = time.time() - start_time


# BFS Execution (ChatGpt)

start_time = time.time()
bfs_path, bfs_nodes = bfs(graph_ex, start, goal)
bfs_time = time.time() - start_time

# A* Execution
start_time = time.time()
astar_2_path, astar_2_cost = astar_2(graph_ex, start, goal)
astar_2_time = time.time() - start_time

# Results
# Affichage des résultats

print("BFS \n")
if bfs_path:
    print(f"Le coût pour atteindre le but est : {bfs_nodes}")  # Afficher le coût du chemin
    print(f"Le chemin suivi est : {' -> '.join(map(str, bfs_path))}")  # Afficher le chemin suivi
    print(f"temps = {bfs_time}")  # Afficher le temps d'exécution
else:
    print("Aucun chemin trouvé.")  # Si aucun chemin n'a été trouvé

print("\n")
print("-------------------------------------------------------------------------------")
print("\n")

print("A* 1 \n")
if astar_1_cost:
    print(f"Le coût pour atteindre le but est : {astar_1_cost}")  # Afficher le coût du chemin
    print(f"Le chemin suivi est : {' -> '.join(map(str, astar_1_path))}")  # Afficher le chemin suivi
    print(f"temps = {astar_1_time}")  # Afficher le temps d'exécution
else:
    print("Aucun chemin trouvé.")  # Si aucun chemin n'a été trouvé
    
print("\n")
print("-------------------------------------------------------------------------------")
print("\n")

print("A* 2\n")
if astar_2_path:
    print(f"Le coût pour atteindre le but est : {astar_2_cost}")  # Afficher le coût du chemin
    print(f"Le chemin suivi est : {' -> '.join(map(str, astar_2_path))}")  # Afficher le chemin suivi
    print(f"temps = {astar_2_time}")  # Afficher le temps d'exécution
else:
    print("Aucun chemin trouvé.")  # Si aucun chemin n'a été trouvé


BFS 

Le coût pour atteindre le but est : 7
Le chemin suivi est : (0, 0) -> (0, 1) -> (1, 1) -> (1, 2) -> (1, 3) -> (1, 4) -> (2, 4) -> (3, 4)
temps = 0.00027942657470703125


-------------------------------------------------------------------------------


A* 1 

Le coût pour atteindre le but est : 7
Le chemin suivi est : (0, 0) -> (1, 0) -> (1, 1) -> (1, 2) -> (1, 3) -> (1, 4) -> (2, 4) -> (3, 4)
temps = 0.000713348388671875


-------------------------------------------------------------------------------


A* 2

Le coût pour atteindre le but est : 7
Le chemin suivi est : (0, 0) -> (0, 1) -> (1, 1) -> (1, 2) -> (1, 3) -> (1, 4) -> (2, 4) -> (3, 4)
temps = 0.0002474784851074219


## Remarque 
Comme discuté en TP02, bfs est optimal. Nous voyons que A* 1 et A* 2 le sont aussi.
Au niveau de la comparaison entre ces trois algorithmes: On a que A* 2 est plus performant que BFS qui est plus performant que A* 1.
Ainsi on veut que d'avoir une heuristique et un triage de queue efficace est plus performant qu'un simple BFS. 

# Ex 2 : 
Pour notre projet, nous devons appliquer les algorithme étudiés, les algorithme de recherche,  à un problème. 

### Définition du problème :
Vous êtes un étudiant, en cours d’informatique, et vous devez solliciter l’expertise d’un assistant. Vous avez une pause de 15 minutes à disposition et vous devez trouver le chemin le plus court pour avoir le plus de temps possible avec l’assistant.

L’université est vue comme un ensemble d’intersections de couloirs (des nœuds) connectées par des couloirs (pondérés par leur distance). Le coût associé à chaque traversée de couloir correspond au temps qu’il a fallu utiliser pour le traverser.

Cependant, il se trouve réparties dans l’université des machines à café qui, en consommant un café, nous rendent plus efficace, comme si nous gagnions 10 minutes. Toutefois, nous ne pouvons pas utiliser chaque machine à café plus d’une fois sans risquer une overdose de café.


(Nous pourrions par la suite intégrer des variations au problème, par exemple avec des événements probabilistes apparaissant dans l’université, pour déterminer par exemple quand nous devrions continuer le projet avec des chaînes de Markov.)

In [14]:
# Graphe de l'université avec intersections, couloirs pondérés et machines à café
university_graph = {
    (0, 0): [((0, 1), 2), ((1, 0), 3)],
    (0, 1): [((0, 0), 2), ((0, 2), 3), ((1, 1), 2)],
    (0, 2): [((0, 1), 3), ((0, 3), 1), ((1, 2), 2)],
    (0, 3): [((0, 2), 1), ((0, 4), 2), ((1, 3), 3)],  # Machine à café
    (0, 4): [((0, 3), 2), ((1, 4), 3)],
    (1, 0): [((0, 0), 3), ((1, 1), 2), ((2, 0), 3)],  
    (1, 1): [((1, 0), 2), ((0, 1), 2), ((1, 2), 1), ((2, 1), 2)],
    (1, 2): [((1, 1), 1), ((0, 2), 2), ((1, 3), 2), ((2, 2), 2)],
    (1, 3): [((1, 2), 2), ((0, 3), 3), ((1, 4), 1), ((2, 3), 2)],
    (1, 4): [((1, 3), 1), ((0, 4), 3), ((2, 4), 2)],  # Machine à café
    (2, 0): [((1, 0), 3), ((2, 1), 1)],
    (2, 1): [((2, 0), 1), ((1, 1), 2), ((2, 2), 2), ((3, 1), 2)],
    (2, 2): [((2, 1), 2), ((1, 2), 2), ((2, 3), 1), ((3, 2), 3)],
    (2, 3): [((2, 2), 1), ((1, 3), 2), ((2, 4), 3), ((3, 3), 2)],
    (2, 4): [((2, 3), 3), ((1, 4), 2), ((3, 4), 2)],
    (3, 1): [((2, 1), 2), ((3, 2), 2)],
    (3, 2): [((3, 1), 2), ((2, 2), 3), ((3, 3), 1)],
    (3, 3): [((3, 2), 1), ((2, 3), 2), ((3, 4), 2)],
    (3, 4): [((3, 3), 2), ((2, 4), 2)]  # Bureau de l'assistant
}

# Machines à café
coffee_machines = {(0, 3), (1, 4)}

# Position de départ et d'arrivée
start = (0, 0)
goal = (3, 4)


# A* modifié pour gérer les machines à café
def astar_2(graph, start, goal, coffee_machines):
    open_set = []
    heapq.heappush(open_set, (0, start, frozenset(), [start]))  # (score, position, cafés visités, chemin), nous ajontons les cafés visités via un ensemble immuable
    g_scores = {(start, frozenset()): 0}  # Score de coût minimum atteint avec cet état
    nodes_explored = 0

    while open_set:
        _, current, visited_coffees, path = heapq.heappop(open_set)
        nodes_explored += 1

        if current == goal:
            return path, nodes_explored

        for neighbor, cost in graph.get(current, []):
            new_visited = visited_coffees
            new_cost = g_scores[(current, visited_coffees)] + cost
            
            # Si c'est une machine à café non encore visitée, appliquer le bonus (-10)
            if neighbor in coffee_machines and neighbor not in visited_coffees:
                new_cost -= 10
                new_visited = visited_coffees | {neighbor}  # Ajouter la machine visitée

            state = (neighbor, new_visited)
            if state not in g_scores or new_cost < g_scores[state]:
                g_scores[state] = new_cost
                f_score = new_cost + heuristic(neighbor, goal)
                heapq.heappush(open_set, (f_score, neighbor, new_visited, path + [neighbor]))

    return None, nodes_explored  # Aucun chemin trouvé

# Exécution
start_time = time.time()
astar_cafe_path, astar_cafe_nodes = astar_2(university_graph, start, goal, coffee_machines)
astar_cafe_time = time.time() - start_time


if astar_2_path:
    print(f"Le coût pour atteindre le but est : {astar_cafe_nodes}")  # Afficher le coût du chemin
    print(f"Le chemin suivi est : {' -> '.join(map(str, astar_cafe_path))}")  # Afficher le chemin suivi
    print(f"temps = {astar_cafe_time}")  # Afficher le temps d'exécution
else:
    print("Aucun chemin trouvé.")  # Si aucun chemin n'a été trouvé


Le coût pour atteindre le but est : 12
Le chemin suivi est : (0, 0) -> (0, 1) -> (0, 2) -> (0, 3) -> (0, 4) -> (1, 4) -> (2, 4) -> (3, 4)
temps = 0.0002028942108154297
